In [588]:
#Spencer Tu & Kenneth Kitahata
#02/19/2020
#HW1: Classification and Regression 
import scipy
import numpy as np
import matplotlib.pyplot as plt
import math

In [589]:
training_s1 = np.loadtxt(fname = "http://colindawson.net/data/S1train.csv", delimiter = (","))
test_s1 = np.loadtxt(fname = "http://colindawson.net/data/S1test.csv", delimiter = (","))
training_s2 = np.loadtxt(fname = "http://colindawson.net/data/S2train.csv", delimiter = (","))
test_s2 = np.loadtxt(fname = "http://colindawson.net/data/S2test.csv", delimiter = (","))

In [590]:
#1a(i)
#a and b are an array of numbers representing 2 vectors
#calculates the euclidean disance between them
def euclidean(vector1, vector2):
    distance = np.sqrt(np.dot(vector1, vector1) - 2 * np.dot(vector1, vector2) + np.dot(vector2, vector2))
    return distance

In [591]:
#1a(ii)
#a is the case array (your target)
#b is an N x 2 matrix (your training set)
#calculate distance between xnew and each point in training data, return array of distances
def distance(point, training):
    newtrain = training[:,1:3]
    dist_array = np.zeros([len(training)])
    for i in range(0, len(training)):
        dist_array[i] = (euclidean(point, newtrain[i]))
    return dist_array

In [592]:
#1a(iii)
# return the indicies of K closest neighbors
def closestneighbors(distance, K):
    return np.argsort(distance)[:K]

In [593]:
#1a(iv)
#this function returns the "majority class" of k closest neighbors
# returns either 1, -1 or -100 in case of ties
def majority(training, miniindex):
    counter = np.zeros(2)
    for i in range (len(miniindex)):
        if training[miniindex[i], 0] == 1:
            counter[0] += 1
        else:
            counter[1] += 1
    if counter[0] > counter[1]:
        return 1
    elif counter[1] > counter [0]:
        return -1
    else:
        return -100

In [594]:
#1a(v)
# Assuming xnew is a set of instances and it is in Nx2 matrix
# Assuming training is a training set and it is in Nx3 matrix
# return a vector of class predictions (1 or -1) for xnew

#how to deal with ties
def KNN(xnew, train, k):
    labels = np.zeros([len(xnew)])
    for i in range(0, len(xnew)):
        dist = distance(xnew[i, 1:], train)
        neighbors = closestneighbors(dist, k)
        label = majority(train, neighbors)
        
        #if tie, run again with k - 1 neighbors
        while label == -100:
            neighbors = closestneighbors(dist, k - 1)
            label = majority(train, neighbors) 
            
        labels[i] = majority(train, neighbors)    
    return labels

In [595]:
#1b(i)
def error(label, true):
    num = 0
    denom = len(label)
    for i in range(0, len(label)):
        if label[i] == true[i]:
            num += 1
    return num / denom

In [596]:
#1b(ii)
def performance(name, traindata, xnew, labels, metric, k):
    if name == "KNN":
        predictions = KNN(xnew, traindata, k)
        if metric == "accuracy":
            score = error(predictions, labels)
            return score
    else:
        print('In Progress')

In [597]:
#1b(iii)
#return an array of performance scores for each fold
# if "YES" to generalization, return an array of array of performane scores, array of generalization error, and 
#lastly an array of training error

def CV(data, folds, k, name, metric, generalization): 
    np.random.shuffle(data)
    datainfolds = np.array_split(data, folds)
    generalization_error = np.array([])
    
    #array with performance of classifier for each fold
    classifier_performance = np.array([])
    length = list(range(folds))
    #iterate through each fold 
    for i in length:
        train = np.empty_like(datainfolds[0])
        index = length.copy()
        del index[i]
        for j in index:      
            train = np.append(train, datainfolds[j], axis = 0)
        cut = len(datainfolds[0])
        train = np.delete(train, slice(0, cut), axis = 0)
        validate = datainfolds[i]
        label = validate[:, 0]
        print(train)
        print(validate)
        print(label)
        score = performance(name, train, validate, label, metric, k)
        classifier_performance = np.append(classifier_performance, score)
    
        if generalization == "Yes":
            g_labels = train[:, 0]
            trainerror = performance(name, train, train, g_labels, metric, k)
            generalizationerror = score - trainerror
            generalization_error = np.append(generalization_error, generalizationerror)

    result = np.append(classifier_performance, generalization_error)

    if generalization == "Yes":
        for i in range(folds):
            result = np.append(result, (result[i] - result[i + folds]))
    return result
    #return generalization_error

In [598]:
#1b(iv)
#return classifier performance across each fold
#option in function CV to also return training and generalization errror
def foldperformance(scores, folds):

    classifiermean = np.mean(scores[: folds])
    twentyfive = np.percentile(scores[: folds], 25)
    seventyfive = np.percentile(scores[: folds], 75)
    
    if len(scores) > folds:
        generalizationmean = np.mean(scores[folds:2*folds])
        trainingmean = np.mean(scores[2*folds:])
        print('mean generalization error:', generalizationmean)
        print('mean training error:', trainingmean)
    
    
    print('mean classifier performance:', classifiermean)
    print('25th percentile of classifier:', twentyfive)
    print('75th percentile of classifier:', seventyfive)

In [393]:
#run CV for 10 folds and create graph of performance across folds
for i in range(1, 15, 2):
    scores = CV(training_s1, 10, i, 'KNN', 'accuracy', 'Yes')
    print(scores)
    performance = foldperformance(scores, 10)
    print(performance)

/Users/kennethkitahata/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.90909091 0.90909091 0.90909091 0.90909091
 0.90909091 0.90909091 0.90909091 0.90909091 0.90909091 0.90909091]
mean generalization error: 0.09090909090909094
mean training error: 0.909090909090909
mean classifier performance: 1.0
25th percentile of classifier: 1.0
75th percentile of classifier: 1.0
None


TypeError: 'NoneType' object is not callable